# Pytorch とは

オブジェクト指向型の深層学習ライブラリ。
テンソル計算と自動微分をサポートするフレームワークで、PyTorch ではこれを使って機械学習のモデルやアルゴリズムを構築する。


In [77]:
# Pytorch
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

# Utilities
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List
from sklearn.utils import shuffle
from sklearn import datasets
from sklearn.model_selection import train_test_split

%matplotlib inline

# ネットワークの作成


## nn.Sequential を使った書き方


In [78]:
# ランダムシードの設定
torch.manual_seed(42)

# 簡単なネットワークの設定
model = nn.Sequential(
    nn.Linear(3, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

# ネットワーク構成の確認
model

Sequential(
  (0): Linear(in_features=3, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [79]:
# ネットワークのパラメータの確認
for idx, param in enumerate(model.named_parameters()):  # named_parameters()メソッドはmodelのパラメータを返すgenerator
  print(f'param_{idx}:, {param}')

param_0:, ('0.weight', Parameter containing:
tensor([[ 0.4414,  0.4792, -0.1353],
        [ 0.5304, -0.1265,  0.1165],
        [-0.2811,  0.3391,  0.5090],
        [-0.4236,  0.5018,  0.1081],
        [ 0.4266,  0.0782,  0.2784],
        [-0.0815,  0.4451,  0.0853],
        [-0.2695,  0.1472, -0.2660],
        [-0.0677, -0.2345,  0.3830]], requires_grad=True))
param_1:, ('0.bias', Parameter containing:
tensor([-0.4557, -0.2662, -0.1630, -0.3471,  0.0545, -0.5702,  0.5214, -0.4904],
       requires_grad=True))
param_2:, ('2.weight', Parameter containing:
tensor([[ 0.2730,  0.0588, -0.1148,  0.2185,  0.0551,  0.2857,  0.0387, -0.1115]],
       requires_grad=True))
param_3:, ('2.bias', Parameter containing:
tensor([0.0950], requires_grad=True))


## カスタムレイヤーについて


nn.Module を継承することで OOP に基づいたネットワーク層の記述ができる。


In [80]:
class SimpleClassification(nn.Module):
  """
  Simple classification model consists of two layers with ReLU.
  The input size can be changed and the output size is 3.
  """
  def __init__(self, n_in):
    super().__init__()
    # 一層目の定義
    self.layer1 = nn.Sequential(
      nn.Linear(n_in, 10),
      nn.ReLU()
    )
    # 二層目の定義
    self.layer2 = nn.Sequential(
      nn.Linear(10, 10),
      nn.ReLU()
    )
    
    # 三層目の定義
    self.layer3 = nn.Sequential(
      nn.Linear(10, 3),
    )

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    out = self.layer3(x)
    return out

In [81]:
# 入出力の確認
torch.manual_seed(123)
input = torch.randn(1, 3)

model = SimpleClassification(n_in=len(input[0]))
output = model(input)
print(output.shape)

torch.Size([1, 3])


## なぜ forward メソッドで予測ができるのか


Python では、クラスに特殊メソッド**call**を定義する事でクラスを関数のように使う事ができる。
上記のカスタムクラスで継承している親クラス torch.nn.Module ではこの**call**メソッドを定義しており、その中で forward メソッドが呼ばれる事で処理が走っている。


# 誤差逆伝播


## 誤差逆伝播とは

ニューラルネットワークの学習とは、お手本となる教師データを使ってネットワークのパラメータを調整することである。その過程で出力と教師データの誤差を算出し、その勾配を使ってパラメータを更新する。

その際、出力から入力の方向へ更新が進められるためこの方法を誤差逆伝播と呼ぶ。


In [82]:
# Iris データセットの読み込み
iris = datasets.load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.25 ,random_state=0)

x = torch.tensor(X_train,dtype = torch.float)
y = torch.tensor(y_train)

### 学習のサイクル

学習過程では、model(input)の出力を output とすると、まず損失関数 criterion(output)を計算する(loss)。
この loss から勾配を計算し、optimizer で指定した最適化手法のもと誤差逆伝播を行いパラメータを更新する。


In [83]:
# 学習モデルのインスタンスを作成
model = SimpleClassification(X_train.shape[1])
optimizer = optim.SGD(model.parameters(), lr=0.01)
# 損失関数の定義
criterion = nn.CrossEntropyLoss()

In [84]:
# 学習ループ
epochs = 10
iters = len(x)
for epoch in range(1, epochs):
    for i in range(1, iters):
        # 勾配の初期化
        optimizer.zero_grad()
        # 説明変数xをネットワークにかける
        output = model(x)
        # 損失関数の計算
        loss = criterion(output, y)
        # 勾配の計算 
        loss.backward()
        # パラメタの更新
        optimizer.step()

        if i == len(x)-1:  # 1epochごとにloss表示
            print(f"loss : {loss.item()}")

loss : 1.0081254243850708
loss : 0.8493548631668091
loss : 0.7042444944381714
loss : 0.5836213231086731
loss : 0.48427101969718933
loss : 0.4180900454521179
loss : 0.3666849434375763
loss : 0.31534937024116516
loss : 0.26163241267204285


In [85]:
# 推論評価
outputs = model(torch.tensor(X_test, dtype = torch.float))
_, predicted = torch.max(outputs.data, 1)
y_predicted = predicted.numpy()
accuracy = 100 * np.sum(predicted.numpy() == y_test) / len(y_predicted)
print('accuracy: {:.1f}%'.format(accuracy))

accuracy: 94.7%


また、今回はループをシンプルなものにしたが、通常はミニバッチ学習という手法を使う。
ミニバッチ学習では学習データを任意の小さなまとまり(ミニバッチ)に分割し、このまとまりを 1 単位(1iteration)としてパラメータ更新を行う。
ちなみに、データセット全てを 1 セットとして見たときに何セット学習したかを epoch 数(学習回数)と呼ぶ

[参考資料](https://zenn.dev/nekoallergy/articles/ml-basic-epoch)


## データの扱いについて

データの前処理とバッチ処理は、PyTorch でニューラルネットワークモデルを訓練する際の重要なステップである。

このプロセスを効率化し、柔軟性を提供するために、PyTorch は torch.utils.data.Dataset と torch.utils.data.DataLoader の 2 つの強力なクラスを提供している。これらを使用することで、大規模なデータセットを扱いやすくし、訓練プロセスを効率的に行うことができる。


### Dataset クラス

特徴量行列（ラベル以外のデータ）X とラベル y を TensorDataset というクラスに渡して、特徴量行列とラベルを一つのデータベース的なものにまとめる事ができる。
PyTorch では、この形式で Data を扱うのが基本となる。


#### Pandas から TensorDataset へ変換


In [ ]:
# Load Iris
import seaborn as sns
iris = sns.load_dataset('iris')
print(type(iris))  # Pandasで読込まれている事の確認
iris.head()

In [ ]:
# カテゴリ変数を変換
iris.loc[:, 'species'] = iris.loc[:, 'species'].map({'setosa':0, 'versicolor':1, 'virginica':2})
iris.head()

##### Pandas -> Numpy -> Tensor

Pandas データセットは、直接データセットに変更できないので、Pandas を Numpy 配列に変換してから Dataset 型に変換を行う


In [ ]:
# Numpy変換の為にvaluesというメソッドを使用

X = torch.FloatTensor(iris.drop("species", axis=1).values)
y = torch.LongTensor(iris["species"].values.astype(np.int64))  # dtypeをint64へ変換してから処理

In [ ]:
# Datasetを作成
Dataset = torch.utils.data.TensorDataset(X, y)

In [ ]:
# 結果を確認
X_sample, y_sample = Dataset[100]
print(X_sample, y_sample)

ここまでが、基本的な Dataset の作成方法となる。
一方でこのような Dataset の扱い方は現場ではほとんどの場合なく、もっと便利な自作（カスタム）Dataset というものを作成する。


#### カスタム Dataset について


#


##### Dataset の設定


自作 Dataset を使用することで、前処理（Transformer）等を細かくカスタマイズすることができる。
作成の際は以下の 3 点を設定する。

1. torch.utils.data.Dataset クラスを継承
2. 特殊メソッド**len**()の設定
3. 特殊メソッド**getitem**()の設定

以下、実際にカスタム Dataset の定義を行う。


[参考](../OOP.ipynb)


In [ ]:
class IrisDataset(torch.utils.data.Dataset):
    def __init__(self, df: pd.DataFrame, features: list[str], labels: list[str]):
        self.features = df[features].values
        self.labels = df[labels].values.astype(np.int64)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        feature = torch.FloatTensor(self.features[idx])
        label = torch.LongTensor(self.labels[idx])
        return feature, label

In [ ]:
# インスタンス化
iris_dataset = IrisDataset(iris, ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], ['species'])

In [ ]:
# __len__メソッドの確認
len(iris_dataset)

In [ ]:
# __getitem__メソッドの確認
iris_dataset[100]

##### 前処理の追加


任意の処理を施す為の
